## Fine tuning with Distill HUBERT

In [1]:
import gradio as gr
import numpy as np
import evaluate

from datasets import load_dataset, Audio
from transformers import AutoFeatureExtractor
from transformers import AutoModelForAudioClassification
from transformers import TrainingArguments
from transformers import Trainer

In [2]:
gtzan = load_dataset("marsyas/gtzan", "all")
gtzan

Found cached dataset gtzan (/home/.cache/huggingface/datasets/marsyas___gtzan/all/0.0.0/8bd0e23c2d9b2be30d36bc6834319772dff22a3bd28527996612386cef003910)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 999
    })
})

In [3]:
gtzan = gtzan["train"].train_test_split(seed=42, shuffle=True, test_size=0.1)
gtzan

Loading cached split indices for dataset at /home/.cache/huggingface/datasets/marsyas___gtzan/all/0.0.0/8bd0e23c2d9b2be30d36bc6834319772dff22a3bd28527996612386cef003910/cache-52d2398c8e4ac745.arrow and /home/.cache/huggingface/datasets/marsyas___gtzan/all/0.0.0/8bd0e23c2d9b2be30d36bc6834319772dff22a3bd28527996612386cef003910/cache-3bcc56e346e4d81c.arrow


DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})

In [4]:
id2label_fn = gtzan["train"].features["genre"].int2str
id2label_fn(gtzan["train"][0]["genre"])

'pop'

In [5]:
# def generate_audio():
#     example = gtzan["train"].shuffle()[0]
#     audio = example["audio"]
#     return (
#         audio["sampling_rate"],
#         audio["array"],
#     ), id2label_fn(example["genre"])


# with gr.Blocks() as demo:
#     with gr.Column():
#         for _ in range(2):
#             audio, label = generate_audio()
#             output = gr.Audio(audio, label=label)

# demo.launch(debug=True)

In [6]:
model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

In [7]:
sampling_rate = feature_extractor.sampling_rate
sampling_rate

16000

In [8]:
gtzan = gtzan.cast_column("audio", Audio(sampling_rate=sampling_rate))

In [9]:
gtzan["train"][0]

{'file': '/home/.cache/huggingface/datasets/downloads/extracted/b96ff32d83db7594c836f58f0a9202aa3f903426f5e00926246a317fa68f2686/genres/pop/pop.00098.wav',
 'audio': {'path': '/home/.cache/huggingface/datasets/downloads/extracted/b96ff32d83db7594c836f58f0a9202aa3f903426f5e00926246a317fa68f2686/genres/pop/pop.00098.wav',
  'array': array([ 0.0873509 ,  0.20183384,  0.4790867 , ..., -0.18743178,
         -0.23294401, -0.13517427]),
  'sampling_rate': 16000},
 'genre': 7}

In [10]:
sample = gtzan["train"][0]["audio"]

print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

Mean: 0.000185, Variance: 0.0493


In [11]:
inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])

print(f"inputs keys: {list(inputs.keys())}")

print(
    f"Mean: {np.mean(inputs['input_values']):.3}, Variance: {np.var(inputs['input_values']):.3}"
)

inputs keys: ['input_values', 'attention_mask']
Mean: -7.45e-09, Variance: 1.0


In [12]:
max_duration = 30.0


def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

In [13]:
gtzan_encoded = gtzan.map(
    preprocess_function, remove_columns=["audio", "file"], batched=True, num_proc=1
)
gtzan_encoded

Loading cached processed dataset at /home/.cache/huggingface/datasets/marsyas___gtzan/all/0.0.0/8bd0e23c2d9b2be30d36bc6834319772dff22a3bd28527996612386cef003910/cache-12773189444aee63.arrow
Loading cached processed dataset at /home/.cache/huggingface/datasets/marsyas___gtzan/all/0.0.0/8bd0e23c2d9b2be30d36bc6834319772dff22a3bd28527996612386cef003910/cache-de2b194a98a4af18.arrow


DatasetDict({
    train: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 899
    })
    test: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 100
    })
})

In [14]:
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")

In [15]:
id2label = {
    str(i): id2label_fn(i)
    for i in range(len(gtzan_encoded["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

id2label["7"]

'pop'

## Fine-tuning

In [16]:
num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['projector.bias', 'projector.weight', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
model_name = model_id.split("/")[-1]
batch_size = 8
gradient_accumulation_steps = 5
num_train_epochs = 5

training_args = TrainingArguments(
    f"hfa-{model_name}-finetuned-gtzan-exercise",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-3,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=True,
)

In [22]:
metric = evaluate.load("accuracy")

In [23]:
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [24]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

Cloning https://huggingface.co/kurianbenoy/hfa-distilhubert-finetuned-gtzan-exercise into local empty directory.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
0,1.969300,1.681736,0.410000
1,1.368300,0.964450,0.670000
2,1.042500,0.982123,0.680000
3,0.831900,0.998065,0.690000
4,0.533600,0.809713,0.790000


TrainOutput(global_step=110, training_loss=1.2174951336600564, metrics={'train_runtime': 2470.2407, 'train_samples_per_second': 1.82, 'train_steps_per_second': 0.045, 'total_flos': 2.988528213888e+17, 'train_loss': 1.2174951336600564, 'epoch': 4.87})

In [25]:
kwargs = {
    "dataset_tags": "marsyas/gtzan",
    "dataset": "GTZAN",
    "model_name": f"{model_name}-finetuned-gtzan",
    "finetuned_from": model_id,
    "tasks": "audio-classification",
}

In [26]:
trainer.push_to_hub(**kwargs)

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/90.4M [00:00<?, ?B/s]

To https://huggingface.co/kurianbenoy/hfa-distilhubert-finetuned-gtzan-exercise
   a8130a9..a5afcf9  main -> main

